***** Model Training *********

In [4]:
import pandas as pd
from  sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import geopy
from geopy.distance import geodesic

In [35]:
df= pd.read_csv(r"C:/Zomato delivery/notebook/data/finalTrain.csv")
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [6]:
def calculate_distance(row):
    restaurant_coords = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    delivery_coords = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    return geodesic(restaurant_coords, delivery_coords).kilometers


df['Distance (in KM)'] = df.apply(calculate_distance, axis=1)

df.head()


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Distance (in KM)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,...,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,10.271464
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,...,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,6.229376
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,...,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,13.764306
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,...,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,2.927795
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,...,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,19.373484


In [7]:
df.drop(columns=["ID","Delivery_person_ID","Restaurant_latitude","Restaurant_longitude","Delivery_location_latitude","Delivery_location_longitude","Order_Date","Time_Orderd","Time_Order_picked","Type_of_order"],axis=1,inplace=True)


In [8]:
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Distance (in KM)
0,36.0,4.2,Fog,Jam,2,motorcycle,3.0,No,Metropolitian,46,10.271464
1,21.0,4.7,Stormy,High,1,motorcycle,1.0,No,Metropolitian,23,6.229376
2,23.0,4.7,Sandstorms,Medium,1,scooter,1.0,No,Metropolitian,21,13.764306
3,34.0,4.3,Sandstorms,Low,0,motorcycle,0.0,No,Metropolitian,20,2.927795
4,24.0,4.7,Fog,Jam,1,scooter,1.0,No,Metropolitian,41,19.373484


In [10]:
X=df.drop('Time_taken (min)',axis=1)
y=df["Time_taken (min)"]

In [19]:
categorical_cols=X.select_dtypes(include="object").columns
numerical_cols=X.select_dtypes(exclude="object").columns

print (categorical_cols)
print (numerical_cols)

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_vehicle',
       'Festival', 'City'],
      dtype='object')
Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries', 'Distance (in KM)'],
      dtype='object')


In [12]:
y

0        46
1        23
2        21
3        20
4        41
         ..
45579    32
45580    36
45581    16
45582    26
45583    36
Name: Time_taken (min), Length: 45584, dtype: int64

In [30]:
weather_condition_map=["Sunny","Stormy","Sandstorms","Windy","Fog","Cloudy"]
road_traffic_map=["Low","Medium","High","Jam"]
type_of_vehicle_map=["electric_scooter", "scooter","bicycle","motorcycle"]
festival_map=["No","Yes"]
city_map=["Urban","Metropolitian", "Semi-Urban"]

In [31]:
##Numerical Pipelines

num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

##Categorical Pipeline

cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[weather_condition_map,road_traffic_map,type_of_vehicle_map,festival_map,city_map])),
    ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer( [
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])


In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.40,random_state=32)

In [34]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train) , columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test) , columns=preprocessor.get_feature_names_out())

In [37]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__Distance (in KM),cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City
0,0.955117,1.107922,1.164360,0.43396,-0.083528,1.448825,1.309537,-1.854557,-0.142056,2.887445
1,-0.626784,0.193624,-0.028748,-1.32438,-0.087329,0.270305,0.508539,0.834466,-0.142056,-1.847490
2,0.955117,1.107922,1.164360,0.43396,-0.091403,0.270305,-1.093458,-0.958216,-0.142056,0.519977
3,-1.681384,1.107922,1.164360,0.43396,-0.082185,0.859565,1.309537,0.834466,-0.142056,0.519977
4,0.427817,-0.111142,-1.221856,0.43396,-0.080918,0.859565,-0.292460,0.834466,-0.142056,0.519977
...,...,...,...,...,...,...,...,...,...,...
27345,1.306651,0.193624,1.164360,-1.32438,-0.075372,1.448825,1.309537,-0.958216,-0.142056,0.519977
27346,1.658184,0.193624,-1.221856,0.43396,-0.088776,0.270305,-0.292460,0.834466,-0.142056,0.519977
27347,3.591619,4.155582,2.357468,0.43396,16.282723,0.859565,-1.093458,-0.061875,-0.142056,0.519977
27348,0.076283,-0.111142,-1.221856,2.19230,-0.078866,-0.908214,1.309537,0.834466,-0.142056,0.519977


In [42]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 640.0 kB/s eta 0:02:36
   ---------------------------------------- 0.1/99.8 MB 1.0 MB/s eta 0:01:37
   ---------------------------------------- 0.4/99.8 MB 3.1 MB/s eta 0:00:32
   ---------------------------------------- 0.7/99.8 MB 4.6 MB/s eta 0:00:22
   ---------------------------------------- 0.9/99.8 MB 5.0 MB/s eta 0:00:20
   ---------------------------------------- 1.1/99.8 MB 4.5 MB/s eta 0:00:22
    --------------------------------------- 1.3/99.8 MB 4.4 MB/s eta 0:00:23
    --------------------------------------- 1.4/99.8 MB 4.5 MB/s eta 0:00:22
    ---------------------------------

In [43]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

In [39]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [48]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'RandomForestRegressor':RandomForestRegressor(),
    'XGBRegressor':XGBRegressor(),
    'SVR':SVR()

}

model_list=[] 
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

    


LinearRegression
Model Training Performance
RMSE: 6.447030674701173
MAE: 5.153455842942398
R2 score 52.62401844411412


Lasso
Model Training Performance
RMSE: 6.877129999520747
MAE: 5.517833710261398
R2 score 46.09200049028049


Ridge
Model Training Performance
RMSE: 6.447030225494602
MAE: 5.153457369466933
R2 score 52.6240250460996


Elasticnet
Model Training Performance
RMSE: 6.970894234031697
MAE: 5.611302555870659
R2 score 44.611993471156296


RandomForestRegressor
Model Training Performance
RMSE: 4.064176296583253
MAE: 3.2011588424554858
R2 score 81.17288284848364


XGBRegressor
Model Training Performance
RMSE: 3.9681825456506816
MAE: 3.1566441577163666
R2 score 82.05175319604093


SVR
Model Training Performance
RMSE: 5.61864817103937
MAE: 4.374103367153485
R2 score 64.01658112776252




In [49]:
best_model_index = r2_list.index(max(r2_list))
best_model = model_list[best_model_index]
best_r2_score = r2_list[best_model_index]

print("Best Model based on R2 Score:", best_model)
print("Best R2 Score:", best_r2_score * 100)


Best Model based on R2 Score: XGBRegressor
Best R2 Score: 82.05175319604093
